<a href="https://colab.research.google.com/github/ayeshamaqsood6100-lab/My-Projects-/blob/main/Version_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title 🚀 Meta Sapiens: HIGH QUALITY (1B Models)
# ============================================================================
# META SAPIENS FOR PSL - HIGH QUALITY EDITION
# ============================================================================
# Uses the larger 1B models for maximum accuracy:
# - POSE 1B: Best keypoint accuracy
# - SEG 1B: Best segmentation quality
# ============================================================================

import os
import sys
import subprocess
import json

# ============================================================================
# PHASE 0: INSTALL DEPENDENCIES
# ============================================================================
print("🔧 PHASE 0: INSTALLING DEPENDENCIES")
print("-" * 60)

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])

print("📦 Installing dependencies...")
install("torch")
install("torchvision")
install("opencv-python")
install("tqdm")
install("huggingface_hub")

print("✅ Dependencies installed!")
print()

# ============================================================================
# PHASE 1: MOUNT GOOGLE DRIVE
# ============================================================================
print("🚀 PHASE 1: MOUNTING GOOGLE DRIVE")
print("-" * 60)

try:
    from google.colab import drive
    if not os.path.exists('/content/drive'):
        drive.mount('/content/drive')
    IN_COLAB = True
    print("✅ Google Drive mounted!")
except ImportError:
    IN_COLAB = False
    print("⚠️ Not running in Colab.")

print()

# ============================================================================
# PHASE 2: DOWNLOAD HIGH-QUALITY MODELS (1B)
# ============================================================================
print("🚀 PHASE 2: DOWNLOADING 1B MODELS (Higher Quality)")
print("-" * 60)
print("⚠️ 1B models are larger (~4GB each). This may take a few minutes...")
print()

import torch
import cv2
import numpy as np
from tqdm import tqdm
from huggingface_hub import hf_hub_download

MODEL_DIR = "/content/sapiens_models" if IN_COLAB else "./sapiens_models"

# Download POSE 1B model (highest accuracy)
print("📦 Downloading Sapiens POSE 1B Model...")
POSE_MODEL_PATH = hf_hub_download(
    repo_id="facebook/sapiens-pose-1b-torchscript",
    filename="sapiens_1b_goliath_best_goliath_AP_639_torchscript.pt2",
    local_dir=MODEL_DIR
)
print("✅ Pose 1B model ready!")

# Download SEG 1B model (highest quality)
print("📦 Downloading Sapiens SEG 1B Model...")
SEG_MODEL_PATH = hf_hub_download(
    repo_id="facebook/sapiens-seg-1b-torchscript",
    filename="sapiens_1b_goliath_best_goliath_mIoU_7994_epoch_151_torchscript.pt2",
    local_dir=MODEL_DIR
)
print("✅ Segmentation 1B model ready!")
print()

# ============================================================================
# BODY PART CLASSES & COLORS (28 classes)
# ============================================================================

BODY_PARTS = [
    "Background", "Head", "Neck", "Torso",
    "Left_Upper_Arm", "Right_Upper_Arm", "Left_Lower_Arm", "Right_Lower_Arm",
    "Left_Hand", "Right_Hand", "Left_Upper_Leg", "Right_Upper_Leg",
    "Left_Lower_Leg", "Right_Lower_Leg", "Left_Foot", "Right_Foot",
    "Hair", "Face", "Teeth", "Tongue", "Upper_Lip", "Lower_Lip",
    "Left_Eyebrow", "Right_Eyebrow", "Left_Eye", "Right_Eye", "Nose", "Ears"
]

# Vibrant colors matching Meta's demo
BODY_PART_COLORS = np.array([
    [0, 0, 0],         # Background
    [255, 220, 200],   # Head
    [255, 200, 170],   # Neck
    [100, 200, 100],   # Torso - Green
    [50, 255, 50],     # Left_Upper_Arm - Green
    [255, 165, 0],     # Right_Upper_Arm - Orange
    [0, 255, 128],     # Left_Lower_Arm
    [255, 140, 0],     # Right_Lower_Arm
    [0, 255, 255],     # Left_Hand - Cyan
    [255, 100, 100],   # Right_Hand - Red
    [100, 100, 255],   # Left_Upper_Leg - Blue
    [255, 100, 255],   # Right_Upper_Leg - Purple
    [50, 50, 200],     # Left_Lower_Leg
    [200, 50, 200],    # Right_Lower_Leg
    [0, 200, 200],     # Left_Foot - Teal
    [200, 200, 0],     # Right_Foot - Yellow
    [139, 90, 43],     # Hair - Brown
    [255, 210, 180],   # Face - Skin
    [255, 255, 255],   # Teeth - White
    [255, 100, 150],   # Tongue - Pink
    [255, 150, 150],   # Upper_Lip
    [255, 120, 120],   # Lower_Lip
    [160, 110, 60],    # Left_Eyebrow
    [160, 110, 60],    # Right_Eyebrow
    [100, 160, 220],   # Left_Eye
    [100, 160, 220],   # Right_Eye
    [255, 200, 160],   # Nose
    [200, 160, 110],   # Ears
], dtype=np.uint8)

# ============================================================================
# SKELETON DEFINITIONS
# ============================================================================

GOLIATH_KEYPOINTS = [
    'nose', 'left_eye', 'right_eye', 'left_ear', 'right_ear',
    'left_shoulder', 'right_shoulder', 'left_elbow', 'right_elbow',
    'left_hip', 'right_hip', 'left_knee', 'right_knee',
    'left_ankle', 'right_ankle', 'left_big_toe', 'left_small_toe',
    'left_heel', 'right_big_toe', 'right_small_toe', 'right_heel',
    'right_thumb4', 'right_thumb3', 'right_thumb2', 'right_thumb_third_joint',
    'right_forefinger4', 'right_forefinger3', 'right_forefinger2', 'right_forefinger_third_joint',
    'right_middle_finger4', 'right_middle_finger3', 'right_middle_finger2', 'right_middle_finger_third_joint',
    'right_ring_finger4', 'right_ring_finger3', 'right_ring_finger2', 'right_ring_finger_third_joint',
    'right_pinky_finger4', 'right_pinky_finger3', 'right_pinky_finger2', 'right_pinky_finger_third_joint',
    'right_wrist',
    'left_thumb4', 'left_thumb3', 'left_thumb2', 'left_thumb_third_joint',
    'left_forefinger4', 'left_forefinger3', 'left_forefinger2', 'left_forefinger_third_joint',
    'left_middle_finger4', 'left_middle_finger3', 'left_middle_finger2', 'left_middle_finger_third_joint',
    'left_ring_finger4', 'left_ring_finger3', 'left_ring_finger2', 'left_ring_finger_third_joint',
    'left_pinky_finger4', 'left_pinky_finger3', 'left_pinky_finger2', 'left_pinky_finger_third_joint',
    'left_wrist', 'left_olecranon', 'right_olecranon',
    'left_cubital_fossa', 'right_cubital_fossa', 'left_acromion', 'right_acromion', 'neck',
]
for i in range(len(GOLIATH_KEYPOINTS), 308):
    GOLIATH_KEYPOINTS.append(f'face_kp_{i}')

SKELETON = [
    # Body core (blue)
    ('left_shoulder', 'right_shoulder', (51, 153, 255)),
    ('left_hip', 'right_hip', (51, 153, 255)),
    ('left_shoulder', 'left_hip', (51, 153, 255)),
    ('right_shoulder', 'right_hip', (51, 153, 255)),
    ('neck', 'left_shoulder', (51, 153, 255)),
    ('neck', 'right_shoulder', (51, 153, 255)),
    # Left side (green)
    ('left_shoulder', 'left_elbow', (0, 255, 0)),
    ('left_elbow', 'left_wrist', (0, 255, 0)),
    ('left_hip', 'left_knee', (0, 255, 0)),
    ('left_knee', 'left_ankle', (0, 255, 0)),
    ('left_ankle', 'left_big_toe', (0, 255, 0)),
    ('left_ankle', 'left_heel', (0, 255, 0)),
    # Right side (orange)
    ('right_shoulder', 'right_elbow', (255, 128, 0)),
    ('right_elbow', 'right_wrist', (255, 128, 0)),
    ('right_hip', 'right_knee', (255, 128, 0)),
    ('right_knee', 'right_ankle', (255, 128, 0)),
    ('right_ankle', 'right_big_toe', (255, 128, 0)),
    ('right_ankle', 'right_heel', (255, 128, 0)),
    # Face (blue)
    ('nose', 'left_eye', (51, 153, 255)),
    ('nose', 'right_eye', (51, 153, 255)),
    ('left_eye', 'left_ear', (51, 153, 255)),
    ('right_eye', 'right_ear', (51, 153, 255)),
    # Left hand fingers
    ('left_wrist', 'left_thumb_third_joint', (255, 128, 0)),
    ('left_thumb_third_joint', 'left_thumb2', (255, 128, 0)),
    ('left_thumb2', 'left_thumb3', (255, 128, 0)),
    ('left_thumb3', 'left_thumb4', (255, 128, 0)),
    ('left_wrist', 'left_forefinger_third_joint', (255, 153, 255)),
    ('left_forefinger_third_joint', 'left_forefinger2', (255, 153, 255)),
    ('left_forefinger2', 'left_forefinger3', (255, 153, 255)),
    ('left_forefinger3', 'left_forefinger4', (255, 153, 255)),
    ('left_wrist', 'left_middle_finger_third_joint', (102, 178, 255)),
    ('left_middle_finger_third_joint', 'left_middle_finger2', (102, 178, 255)),
    ('left_middle_finger2', 'left_middle_finger3', (102, 178, 255)),
    ('left_middle_finger3', 'left_middle_finger4', (102, 178, 255)),
    ('left_wrist', 'left_ring_finger_third_joint', (255, 51, 51)),
    ('left_ring_finger_third_joint', 'left_ring_finger2', (255, 51, 51)),
    ('left_ring_finger2', 'left_ring_finger3', (255, 51, 51)),
    ('left_ring_finger3', 'left_ring_finger4', (255, 51, 51)),
    ('left_wrist', 'left_pinky_finger_third_joint', (0, 255, 0)),
    ('left_pinky_finger_third_joint', 'left_pinky_finger2', (0, 255, 0)),
    ('left_pinky_finger2', 'left_pinky_finger3', (0, 255, 0)),
    ('left_pinky_finger3', 'left_pinky_finger4', (0, 255, 0)),
    # Right hand fingers
    ('right_wrist', 'right_thumb_third_joint', (255, 128, 0)),
    ('right_thumb_third_joint', 'right_thumb2', (255, 128, 0)),
    ('right_thumb2', 'right_thumb3', (255, 128, 0)),
    ('right_thumb3', 'right_thumb4', (255, 128, 0)),
    ('right_wrist', 'right_forefinger_third_joint', (255, 153, 255)),
    ('right_forefinger_third_joint', 'right_forefinger2', (255, 153, 255)),
    ('right_forefinger2', 'right_forefinger3', (255, 153, 255)),
    ('right_forefinger3', 'right_forefinger4', (255, 153, 255)),
    ('right_wrist', 'right_middle_finger_third_joint', (102, 178, 255)),
    ('right_middle_finger_third_joint', 'right_middle_finger2', (102, 178, 255)),
    ('right_middle_finger2', 'right_middle_finger3', (102, 178, 255)),
    ('right_middle_finger3', 'right_middle_finger4', (102, 178, 255)),
    ('right_wrist', 'right_ring_finger_third_joint', (255, 51, 51)),
    ('right_ring_finger_third_joint', 'right_ring_finger2', (255, 51, 51)),
    ('right_ring_finger2', 'right_ring_finger3', (255, 51, 51)),
    ('right_ring_finger3', 'right_ring_finger4', (255, 51, 51)),
    ('right_wrist', 'right_pinky_finger_third_joint', (0, 255, 0)),
    ('right_pinky_finger_third_joint', 'right_pinky_finger2', (0, 255, 0)),
    ('right_pinky_finger2', 'right_pinky_finger3', (0, 255, 0)),
    ('right_pinky_finger3', 'right_pinky_finger4', (0, 255, 0)),
]

# ============================================================================
# PHASE 3: RUN INFERENCE
# ============================================================================
print("🚀 PHASE 3: RUNNING HIGH-QUALITY INFERENCE")
print("-" * 60)

VIDEO_FILENAME = "Bandbaja.mp4"
BASE_PATH = "/content/drive/MyDrive" if IN_COLAB else "."
INPUT_VIDEO = os.path.join(BASE_PATH, VIDEO_FILENAME)
OUTPUT_POSE = os.path.join(BASE_PATH, f"HQ_POSE_{VIDEO_FILENAME}")
OUTPUT_SEG = os.path.join(BASE_PATH, f"HQ_SEG_{VIDEO_FILENAME}")
OUTPUT_COMBINED = os.path.join(BASE_PATH, f"HQ_COMBINED_{VIDEO_FILENAME}")
OUTPUT_JSON = os.path.join(BASE_PATH, f"HQ_Keypoints_{VIDEO_FILENAME}.json")

if not os.path.exists(INPUT_VIDEO):
    print(f"❌ Video not found: {INPUT_VIDEO}")
    sys.exit(1)

print(f"📹 Input: {INPUT_VIDEO}")

# Open video
cap = cv2.VideoCapture(INPUT_VIDEO)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = cap.get(cv2.CAP_PROP_FPS) or 30
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

print(f"📊 {width}x{height} @ {fps:.1f} FPS, {total_frames} frames")

ret, test_frame = cap.read()
if not ret:
    print("⚠️ Converting video...")
    cap.release()
    CONV = "/content/conv.mp4" if IN_COLAB else "./conv.mp4"
    subprocess.run(f'ffmpeg -y -i "{INPUT_VIDEO}" -c:v libx264 "{CONV}"', shell=True, capture_output=True)
    INPUT_VIDEO = CONV
    cap = cv2.VideoCapture(INPUT_VIDEO)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS) or 30
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

# Load models
print("📦 Loading 1B models (may take a moment)...")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"   Device: {device}")

pose_model = torch.jit.load(POSE_MODEL_PATH, map_location=device).eval()
seg_model = torch.jit.load(SEG_MODEL_PATH, map_location=device).eval()
print("✅ Models loaded!")

# High quality video writers (use libx264 via ffmpeg pipe for better quality)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out_pose = cv2.VideoWriter(OUTPUT_POSE, fourcc, fps, (width, height))
out_seg = cv2.VideoWriter(OUTPUT_SEG, fourcc, fps, (width, height))
out_combined = cv2.VideoWriter(OUTPUT_COMBINED, fourcc, fps, (width * 2, height))

# Preprocessing
def preprocess(img):
    img = cv2.resize(img, (768, 1024))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32) / 255.0
    img = (img - [0.485, 0.456, 0.406]) / [0.229, 0.224, 0.225]
    return torch.from_numpy(img.transpose(2, 0, 1)).unsqueeze(0).float()

def get_keypoints(heatmaps, w, h):
    kps = {}
    for i, name in enumerate(GOLIATH_KEYPOINTS):
        if i >= heatmaps.shape[0]:
            break
        hm = heatmaps[i]
        idx = np.argmax(hm)
        y, x = np.unravel_index(idx, hm.shape)
        conf = float(hm[y, x])
        kps[name] = {'x': x * w / hm.shape[1], 'y': y * h / hm.shape[0], 'conf': conf}
    return kps

def draw_pose_hq(frame, kps, conf_thresh=0.2):
    h, w = frame.shape[:2]
    # Thicker lines for high quality
    line_thickness = max(2, int(min(w, h) / 300))
    point_radius = max(3, int(min(w, h) / 200))

    # Draw skeleton
    for start, end, color in SKELETON:
        if start in kps and end in kps:
            k1, k2 = kps[start], kps[end]
            if k1['conf'] > conf_thresh and k2['conf'] > conf_thresh:
                pt1 = (int(k1['x']), int(k1['y']))
                pt2 = (int(k2['x']), int(k2['y']))
                if 0 <= pt1[0] < w and 0 <= pt1[1] < h and 0 <= pt2[0] < w and 0 <= pt2[1] < h:
                    cv2.line(frame, pt1, pt2, color, line_thickness, cv2.LINE_AA)

    # Draw keypoints
    for name, kp in kps.items():
        if kp['conf'] > conf_thresh:
            x, y = int(kp['x']), int(kp['y'])
            if 0 <= x < w and 0 <= y < h:
                # Color based on side
                if 'left' in name:
                    color = (0, 255, 0)
                elif 'right' in name:
                    color = (255, 128, 0)
                else:
                    color = (51, 153, 255)
                cv2.circle(frame, (x, y), point_radius, color, -1, cv2.LINE_AA)
                cv2.circle(frame, (x, y), point_radius, (255, 255, 255), 1, cv2.LINE_AA)  # White outline
    return frame

def draw_seg_hq(seg_map, w, h, original_frame, blend_alpha=0.7):
    """High quality segmentation with blending onto original frame."""
    # Create colored segmentation
    seg_colored = BODY_PART_COLORS[seg_map]
    seg_colored = cv2.resize(seg_colored, (w, h), interpolation=cv2.INTER_LINEAR)

    # Blend with original frame for better quality
    mask = (seg_map != 0).astype(np.uint8)
    mask = cv2.resize(mask, (w, h), interpolation=cv2.INTER_LINEAR)
    mask = np.expand_dims(mask, axis=2)

    blended = (seg_colored * blend_alpha + original_frame * (1 - blend_alpha)).astype(np.uint8)
    result = np.where(mask > 0, blended, original_frame)
    return result.astype(np.uint8)

# Process
all_results = []
print()
print("🎬 Processing with HIGH QUALITY 1B models...")
print("   (This will be slower but much more accurate)")
print()

with tqdm(total=total_frames, desc="Frames") as pbar:
    frame_idx = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        tensor = preprocess(frame).to(device)

        # POSE
        with torch.no_grad():
            pose_out = pose_model(tensor)
        heatmaps = (pose_out[0] if isinstance(pose_out, tuple) else pose_out)[0].cpu().numpy()
        kps = get_keypoints(heatmaps, width, height)

        # SEG
        with torch.no_grad():
            seg_out = seg_model(tensor)
        seg_logits = (seg_out[0] if isinstance(seg_out, tuple) else seg_out)[0].cpu().numpy()
        seg_map = np.argmax(seg_logits, axis=0)

        # Draw HQ outputs
        pose_frame = draw_pose_hq(frame.copy(), kps)
        seg_frame = draw_seg_hq(seg_map, width, height, frame.copy())
        combined = np.hstack([pose_frame, seg_frame])

        out_pose.write(pose_frame)
        out_seg.write(seg_frame)
        out_combined.write(combined)

        all_results.append({
            "frame": frame_idx,
            "timestamp": frame_idx / fps,
            "keypoints": {k: {"x": v["x"], "y": v["y"], "confidence": v["conf"]} for k, v in kps.items()}
        })

        frame_idx += 1
        pbar.update(1)

cap.release()
out_pose.release()
out_seg.release()
out_combined.release()

# Save JSON
with open(OUTPUT_JSON, 'w') as f:
    json.dump({
        "video_info": {"filename": VIDEO_FILENAME, "width": width, "height": height, "fps": fps, "frames": frame_idx},
        "models": {"pose": "Sapiens 1B (308 keypoints)", "segmentation": "Sapiens 1B (28 body parts)"},
        "body_parts": BODY_PARTS,
        "frames": all_results
    }, f, indent=2)

print()
print("=" * 60)
print("✅✅✅ HIGH QUALITY PROCESSING COMPLETE! ✅✅✅")
print("=" * 60)
print()
print("📹 OUTPUTS:")
print(f"   POSE: {OUTPUT_POSE}")
print(f"   SEG: {OUTPUT_SEG}")
print(f"   COMBINED: {OUTPUT_COMBINED}")
print(f"   JSON: {OUTPUT_JSON}")